In [1]:
import sys
print sys.executable

import numpy as np
import matplotlib.gridspec as gridspec

from pyuvdata import UVData
from astropy.time import Time
from matplotlib.pyplot import *
matplotlib.rcParams['figure.figsize'] = (14, 9)

//anaconda/envs/root_dev/bin/python


# Read in uvfits file

In [2]:
# Read in uvfits data
UV = UVData()
uvfits_file = '/Users/JBurba/Google Drive/EoR/zen.2457458.35484.xx.HH.uvcU.uvfits'
UV.read_uvfits(uvfits_file)

# Check for the number of spectral windows
# This notebook only supports one spectral window
if UV.Nspws > 1:
    print 'Plotting currently only supported for one spectral window.'

# Options  
Defined user parameters can be used to modify the data plotted and the plot style.

In [10]:
''' Instrument Options '''

# Antennae can be passed as a list of single antenna numbers
antenna_nums = [10,31,64] # returns auto correlations with each antenna passed

# or as a list of tuples containing the desired antennae numbers
ant_pairs_nums = None #[(64,10)] # must be a list of antenna number tuples

# If inplace = False, forces UVData.select(...) to output a new UVData object
inplace = False

# add polarization selection option(s)



''' Plotting Options '''
# Choose plot_type, can be 'abs', 'real', 'imag', or 'phase'
plot_type = 'abs'

# Set custom dmin and dmax
dmin = None
dmax = None

# Force all subplots to use the same min and max (calculated automatically if dmin and dmax are left as None)
# if force_clim = True
force_clim = True

# Choose colormap
cmap = get_cmap('gnuplot')

# Force log10 scale for log_scale = True
log_scale = True

# Remove white space between subplots and use a master colorbar for all subplots
no_white_space = False

# Set size of subplot titles, labels, etc.
FONTSIZE = 16

# Plotting

In [11]:
''' Apply instrument options '''
# Apply select function with above instrument parameters
UV_selected = UV.select(antenna_nums=antenna_nums,ant_pairs_nums=ant_pairs_nums,inplace=inplace)

# Create masked array for plotting using the flag_array from UVData object
masked_data_array = np.ma.masked_array(UV_selected.data_array,UV_selected.flag_array,fill_value = np.NaN)

# For future use when implementing polarization selects
# From https://pyuvdata.readthedocs.io/en/v1.1/uvdata_parameters.html
polarization_dict = {'V' : 4,
                     'U' : 3,
                     'Q' : 2,
                     'I' : 1,
                    'RR' : -1,
                    'LL' : -2,
                    'RL' : -3,
                    'LR' : -4,
                    'XX' : -5,
                    'YY' : -6,
                    'XY' : -7,
                    'YX' : -8}


''' Plotting option checks '''
# Check for a supported plot_type option
if not plot_type in ['abs','real','imag','phase']:
    print 'Please choose a supported plot type'
    
# If plotting phase, force linear scale
if plot_type == 'phase':
    log_scale = False

# If plotting a single baseline, set no_white_space to False for proper formatting
if UV_selected.Nbls == 1:
    no_white_space = False

# If no_white_space, automatically set force_clim=True
if no_white_space:
    force_clim = True


''' Plotting '''
# Determine necessary number of rows/cols for subplots
bls = np.unique(UV_selected.baseline_array)
nrows = int(np.sqrt(len(bls)))
ncols = int(np.ceil(float(len(bls)) / nrows))
nempty = nrows*ncols-UV_selected.Nbls

# Create figure and gridspec object
if fignum_exists(1):
    clf()
fig = figure(1,facecolor='w')
gs = gridspec.GridSpec(nrows,ncols)
plot_ind = [0,0] # [row,col] in gridspec layout
bls_counter = 0 # counter for baseline in each subplot

# Parameters for annotating subplots with baseline information
nchans = UV.Nfreqs
nints = UV_selected.data_array.shape[0]

if no_white_space:
    # Add master labels for all x and y axes
    master_ax = fig.add_subplot(gs[:,:])
    master_ax.set_xlabel('Frequency [channels]',size=FONTSIZE,labelpad=25)
    master_ax.set_ylabel('Time [integrations]',size=FONTSIZE,labelpad=25)
    master_ax.set_xticks([])
    master_ax.set_yticks([])
    master_ax.spines['top'].set_visible(False)
    master_ax.spines['right'].set_visible(False)
    master_ax.spines['bottom'].set_visible(False)
    master_ax.spines['left'].set_visible(False)

# Assign range for colorbar range if dmin and dmax not specified
if dmin is None and dmax is None:
    if log_scale:
        if plot_type == 'abs':
            dmin = np.ma.min(np.log10(np.abs(masked_data_array)))
            dmax = np.ma.max(np.log10(np.abs(masked_data_array)))
        elif plot_type == 'real':
            dmin = np.ma.min(np.log10(np.real(masked_data_array)))
            dmax = np.ma.max(np.log10(np.real(masked_data_array)))
        elif plot_type == 'imag':
            dmin = np.ma.min(np.log10(np.imag(masked_data_array)))
            dmax = np.ma.max(np.log10(np.imag(masked_data_array)))
        else:
            print 'Cannot have log_scale with plot_type= \'phase\''
    else:
        if plot_type == 'abs':
            dmin = np.ma.min(np.abs(masked_data_array))
            dmax = np.ma.max(np.abs(masked_data_array))
        elif plot_type == 'real':
            dmin = np.ma.min(np.real(masked_data_array))
            dmax = np.ma.max(np.real(masked_data_array))
        elif plot_type == 'imag':
            dmin = np.ma.min(np.imag(masked_data_array))
            dmax = np.ma.max(np.imag(masked_data_array))
        else:
            dmin = -np.pi
            dmax = np.pi

# Iterate through gridspec subplot array
while plot_ind[0] < nrows and bls_counter < UV_selected.Nbls:
    while plot_ind[1] < ncols and bls_counter < UV_selected.Nbls:
        ax = fig.add_subplot(gs[plot_ind[0],plot_ind[1]])
        
        # Get indices for given baseline
        bl = bls[bls_counter]
        bls_inds = np.where(UV_selected.baseline_array == bl)[0]
        
        if plot_type == 'abs':
            if log_scale:
                im = ax.imshow(np.log10(np.abs(masked_data_array[bls_inds,0,:,0])),
                               origin='upper',aspect='auto', interpolation='none',
                               cmap=cmap) #,vmin=dmin,vmax=dmax)
            else:
                im = ax.imshow(np.abs(masked_data_array[bls_inds,0,:,0]),
                               origin='upper',aspect='auto', interpolation='none',
                               cmap=cmap) #,vmin=dmin,vmax=dmax)
        elif plot_type == 'real':
            if log_scale:
                im = ax.imshow(np.log10(np.real(masked_data_array[bls_inds,0,:,0])),
                               origin='upper',aspect='auto', interpolation='none',
                               cmap=cmap) #,vmin=dmin,vmax=dmax)
            else:
                im = ax.imshow(np.real(masked_data_array[bls_inds,0,:,0]),
                               origin='upper',aspect='auto', interpolation='none',
                               cmap=cmap) #,vmin=dmin,vmax=dmax)
        elif plot_type == 'imag':
            if log_scale:
                im = ax.imshow(np.log10(np.imag(masked_data_array[bls_inds,0,:,0])),
                               origin='upper',aspect='auto', interpolation='none',
                               cmap=cmap) #,vmin=dmin,vmax=dmax)
            else:
                im = ax.imshow(np.imag(masked_data_array[bls_inds,0,:,0]),
                               origin='upper',aspect='auto', interpolation='none',
                               cmap=cmap) #,vmin=dmin,vmax=dmax)
        else:
            im = ax.imshow(np.angle(masked_data_array[bls_inds,0,:,0]),
                           origin='upper',aspect='auto', interpolation='none',
                           cmap=cmap) #,vmin=dmin,vmax=dmax)
        
        if force_clim:
            # Force plots to same clim
            im.set_clim(dmin,dmax)

        if no_white_space:
            # remove axis labels from interior plots
            if nrows*ncols == UV_selected.Nbls:
                if not plot_ind[0] == nrows-1:
                    ax.set_xticks([])
            else:
                if plot_ind[0] < nrows-2:
                    ax.set_xticks([])
                elif plot_ind[0] == nrows-2 and plot_ind[1] <= ncols-nempty-1:
                    ax.set_xticks([])
            
            if not plot_ind[1] == 0:
                ax.set_yticks([])
            
            # Add antenna numbers to subplot
            ax.text(nchans-50,5,str(UV_selected.baseline_to_antnums(bl)).strip('()'),ha='right',size=FONTSIZE)  
        else:
            # Add title and axis labels to each subplot
            ax.set_title(str(UV_selected.baseline_to_antnums(bl)),size=FONTSIZE)
            ax.set_ylabel('Time [integrations]',size=FONTSIZE)
            ax.set_xlabel('Frequency [channels]',size=FONTSIZE)
            ax.tick_params(axis='both',labelsize=FONTSIZE)
            
            # Add colorbars to subplots
            cb = fig.colorbar(im,ax=ax,format='%.1f')
            cb.ax.tick_params(labelsize=FONTSIZE)
        
        # Update column counter
        plot_ind[1] += 1
        bls_counter += 1
        
    # Reset column counter and update row counter
    plot_ind[1] = 0
    plot_ind[0] +=1

# Update plot dimensions to add master colorbar
gs.tight_layout(fig)

if no_white_space:
    # Append master colorbar
    gs.update(wspace=0.0,hspace=0.0,right=0.85)
    #gs2 = gridspec.GridSpec(1,3)
    #gs2.update(left=0.9,right=1.0,wspace=0.0,hspace=0.0)
    #cbar_ax = fig.add_subplot(gs2[1])
    cbar_ax = fig.add_axes([0.875, 0.15, 0.02, 0.75])
    #cb_ticks = np.linspace(dmin,dmax,10).astype('int')
    cb = fig.colorbar(im, cax=cbar_ax)#,ticks=cb_ticks)
    #cb.ax.set_yticklabels(cb_ticks.astype('str'))
    cb.set_clim(dmin,dmax)
    
    if log_scale:
        cb.set_label('Log Visibilities',size=FONTSIZE,labelpad=10)
    
    cb.ax.tick_params(labelsize=FONTSIZE)
    
show()

divide by zero encountered in log10
divide by zero encountered in log10
divide by zero encountered in log10


In [ ]:
if fignum_exists(1):
    print 'True'





  
  
  
  









------------

# Old plotting code

## Iteration 1

## Iteration 2